In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time


In [2]:
page = 1 

states_dict = {
    1: 'Andaman and Nicobar Islands',
    2: 'Andhra Pradesh',
    3: 'Arunachal Pradesh',
    4: 'Assam',
    5: 'Bihar',
    6: 'Chandigarh',
    7: 'Chhattisgarh',
    8: 'Dadra and Nagar Haveli and Daman and Diu',
    9: 'Delhi',
    10: 'Goa',
    11: 'Gujarat',
    12: 'Haryana',
    13: 'Himachal Pradesh',
    14: 'Jammu and Kashmir',
    15: 'Jharkhand',
    16: 'Karnataka',
    17: 'Kerala',
    18: 'Ladakh',
    19: 'Lakshadweep',
    20: 'Madhya Pradesh',
    21: 'Maharashtra',
    22: 'Manipur',
    23: 'Meghalaya',
    24: 'Mizoram',
    25: 'Nagaland',
    26: 'Odisha',
    27: 'Puducherry',
    28: 'Punjab',
    29: 'Rajasthan',
    30: 'Sikkim',
    31: 'Tamil Nadu',
    32: 'Telangana',
    33: 'Tripura',
    34: 'Uttar Pradesh',
    35: 'Uttarakhand',
    36: 'West Bengal'
}


In [3]:
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

for state in range(1, 37):
    page = 1
    while True:
        url = f"https://myneta.info/LokSabha2024/index.php?action=show_constituencies&state_id={state}&page={page}"
        driver.get(url)

        # Wait until the table is loaded
        try:
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'w3-responsive')))
        except:
            break

        table_elements = driver.find_elements(By.CLASS_NAME, 'w3-responsive')

        for table in table_elements:
            html_content = table.get_attribute("outerHTML")
            
            # Use BeautifulSoup to parse the HTML and remove <script> tags
            soup = BeautifulSoup(html_content, 'html.parser')
            for script in soup(["script", "style"]):
                script.extract()  # Remove the script and style tags
            
            cleaned_html = soup.prettify()

            with open(f"../data/html/{states_dict[state]}_{page}.html", "w", encoding="utf-8") as f:
                f.write(cleaned_html)

        # Check if there is a next page
        try:
            pagination_div = driver.find_element(By.XPATH, "//div[center]/center")
            pagination_soup = BeautifulSoup(pagination_div.get_attribute("outerHTML"), 'html.parser')
            pages_text = pagination_soup.find("small").text.strip()
            total_pages = int(pages_text.split()[-2])

            if page < total_pages:
                page += 1
            else:
                break
        except:
            break

driver.close()